In [ ]:
!pip install ktrain

In [2]:
!wget https://raw.githubusercontent.com/mayanksinghkgp/NLP_Resources/main/FastText_MultiLabel/data/train.csv

--2021-12-22 11:00:29--  https://raw.githubusercontent.com/mayanksinghkgp/NLP_Resources/main/FastText_MultiLabel/data/train.csv
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.108.133, 185.199.109.133, 185.199.110.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.108.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 68802655 (66M) [text/plain]
Saving to: ‘train.csv’

train.csv           100%[===================>]  65.62M  93.6MB/s    in 0.7s    

2021-12-22 11:00:35 (93.6 MB/s) - ‘train.csv’ saved [68802655/68802655]



In [3]:
import pandas as pd
import ktrain
from ktrain import text

In [27]:
PATH = 'train.csv'
NUM_WORDS = 50000     #Number of words in the vocal including ngrams etc
MAXLEN = 150      #Max length of the comment considered

train, val, preproc = text.texts_from_csv(PATH, 'comment_text', label_columns=['toxic', 'severe_toxic', 'obscene', 'threat', 'insult', 'identity_hate'],
                    ngram_range = 2, max_features = NUM_WORDS, maxlen = MAXLEN)

detected encoding: utf-8 (if wrong, set manually)
['toxic', 'severe_toxic', 'obscene', 'threat', 'insult', 'identity_hate']
       toxic  severe_toxic  obscene  threat  insult  identity_hate
43916      0             0        0       0       0              0
32002      0             0        0       0       0              0
25400      0             0        0       0       0              0
49639      0             0        0       0       0              0
2482       0             0        0       0       0              0
['toxic', 'severe_toxic', 'obscene', 'threat', 'insult', 'identity_hate']
        toxic  severe_toxic  obscene  threat  insult  identity_hate
137581      0             0        0       0       0              0
36252       1             0        0       0       0              0
75222       0             0        0       0       0              0
114621      0             0        0       0       0              0
75356       0             0        0       0       0        

In [28]:
text.print_text_classifiers()

fasttext: a fastText-like model [http://arxiv.org/pdf/1607.01759.pdf]
logreg: logistic regression using a trainable Embedding layer
nbsvm: NBSVM model [http://www.aclweb.org/anthology/P12-2018]
bigru: Bidirectional GRU with pretrained fasttext word vectors [https://fasttext.cc/docs/en/crawl-vectors.html]
standard_gru: simple 2-layer GRU with randomly initialized embeddings
bert: Bidirectional Encoder Representations from Transformers (BERT) from keras_bert [https://arxiv.org/abs/1810.04805]
distilbert: distilled, smaller, and faster BERT from Hugging Face transformers [https://arxiv.org/abs/1910.01108]


In [29]:
model = text.text_classifier('fasttext', train, preproc)

Is Multi-Label? True
compiling word ID features...
maxlen is 150
done.


In [30]:
learner = ktrain.get_learner(model, train, val)

In [31]:
learner.autofit(0.001, 3)



begin training using triangular learning rate policy with max lr of 0.001...
Epoch 1/3
4488/4488 [==============================] - 474s 105ms/step - loss: 0.1547 - accuracy: 0.7241 - val_loss: 0.0609 - val_accuracy: 0.9942
Epoch 2/3
4488/4488 [==============================] - 472s 105ms/step - loss: 0.0705 - accuracy: 0.9901 - val_loss: 0.0548 - val_accuracy: 0.9942
Epoch 3/3
4488/4488 [==============================] - 472s 105ms/step - loss: 0.0603 - accuracy: 0.9940 - val_loss: 0.0528 - val_accuracy: 0.9942


In [32]:
predictor  = ktrain.get_predictor(learner.model, preproc)

In [33]:
predictor.predict(['I will make your entire family cry'])

[[('toxic', 0.07829695),
  ('severe_toxic', 0.0001573108),
  ('obscene', 0.010920689),
  ('threat', 0.0009755484),
  ('insult', 0.016092196),
  ('identity_hate', 0.0041721836)]]

In [34]:
predictor.save('/content/toxic_fasttext_bigram')